<a href="https://colab.research.google.com/github/6MrCrazy6/AI_Crypto/blob/main/AI_Crypto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preprocessing

## Importing the libraries

In [3]:
import json
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import random

from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from keras import regularizers

import warnings
warnings.filterwarnings('ignore')

## Importing the dataset



In [4]:
df = pd.read_csv("new_dataset_15k.csv", sep=';', decimal=',', encoding='cp1251')
df.drop(columns=[col for col in df.columns if 'Unnamed:' in col], inplace=True, errors='ignore')

price_cols = ['outcome_price', 'initial_entry_price']
all_potential_feature_cols = [
    col for col in df.columns if
    col.startswith('feat_') or col.startswith('strat_') or col == 'composite_score_value'
]

if 'feat_market_state' in df.columns and df['feat_market_state'].dtype == 'object':
    df['feat_market_state'] = df['feat_market_state'].map(
        {'TREND': 1, 'TRANSITION': 0.5, 'FLAT': 0}
    ).fillna(0)

cols_to_convert_numeric = price_cols + all_potential_feature_cols
for col_name in list(set(cols_to_convert_numeric)):
    if col_name in df.columns:
        df[col_name] = pd.to_numeric(
            df[col_name].astype(str).str.replace(',', '.'), errors='coerce'
        )

df.dropna(subset=price_cols + ['outcome_status'], inplace=True)
if df.empty:
    raise ValueError("DataFrame пуст после очистки NaN в ценах/статусе. Проверьте исходные данные.")

##Feature Engineering

In [5]:
base_feature_columns = [
    col for col in all_potential_feature_cols
    if col in df.columns and pd.api.types.is_numeric_dtype(df[col])
]
if 'feat_market_state' in df.columns and \
   pd.api.types.is_numeric_dtype(df['feat_market_state']) and \
   'feat_market_state' not in base_feature_columns:
    base_feature_columns.append('feat_market_state')
base_feature_columns = list(set(base_feature_columns))

df[base_feature_columns] = df[base_feature_columns].fillna(0)

engineered_feature_columns = base_feature_columns.copy()
lags = [1, 2, 4, 8]
roll_windows = [4, 8]
new_cols_data = {}

for col in base_feature_columns:
    for lag in lags:
        new_cols_data[f'{col}_lag{lag}'] = df[col].shift(lag)
    for window in roll_windows:
        new_cols_data[f'{col}_roll_mean{window}'] = df[col].rolling(window=window, min_periods=1).mean()

df = pd.concat([df, pd.DataFrame(new_cols_data, index=df.index)], axis=1)
df.dropna(inplace=True) # Удаление NaN, появившихся из-за shift/rolling

if df.empty:
    raise ValueError("DataFrame пуст после Feature Engineering и dropna. Проверьте лаги/окна.")

feature_columns = list(set(engineered_feature_columns + list(new_cols_data.keys()))) # Обновляем список всех признаков

## Формирование выборки "сильных движений" и финальных X, y

In [6]:
# ... (Предполагается, что df и feature_columns существуют из Блока 1 и 2) ...

df['price_change'] = (df['outcome_price'] - df['initial_entry_price']) / df['initial_entry_price'] * 100
strong_mask = (((df['outcome_status'] == 'LONG_MET') & (df['price_change'] > 0.5)) |
               ((df['outcome_status'] == 'SHORT_MET') & (df['price_change'] < -0.5)))
strong_moves = df.loc[strong_mask].copy()

if strong_moves.empty:
    raise ValueError("Нет данных после фильтрации 'strong_moves'.")

actual_feature_cols_in_strong = [col for col in feature_columns if col in strong_moves.columns]
X_prepared = strong_moves[actual_feature_cols_in_strong].copy()
X_prepared.fillna(0, inplace=True)

valid_cols = [col for col in actual_feature_cols_in_strong if X_prepared[col].std() > 1e-6]
if not valid_cols:
    raise ValueError("Нет валидных (неконстантных) колонок признаков.")

# Вот здесь определяем X_before_selection и y_labels
X_before_selection = X_prepared[valid_cols].copy()
y_labels = (strong_moves['outcome_status'] == 'LONG_MET').astype(int)

print(f"Размеры перед отбором признаков: X={X_before_selection.shape}, y={y_labels.shape}")

Размеры перед отбором признаков: X=(5101, 259), y=(5101,)


## Splitting the dataset into the Training set and Test set

In [7]:
scaler = StandardScaler()
X_scaled_all_features = scaler.fit_transform(X_before_selection)
X_scaled_all_features = np.nan_to_num(X_scaled_all_features, nan=0.0, posinf=0.0, neginf=0.0)

NUM_FEATURES_TO_SELECT = 80
print(f"Отбор признаков из {X_scaled_all_features.shape[1]} до {NUM_FEATURES_TO_SELECT}...")

selector = SelectFromModel(
    RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
    max_features=NUM_FEATURES_TO_SELECT,
    threshold=-np.inf
)

selector.fit(X_scaled_all_features, y_labels)
X_selected_features = selector.transform(X_scaled_all_features)

selected_feature_indices = selector.get_support(indices=True)
selected_feature_names = X_before_selection.columns[selected_feature_indices].tolist()

print(f"Количество признаков после отбора: {X_selected_features.shape[1]}")
if 0 < X_selected_features.shape[1] <= 10: # Показать имена, если их немного
    print(f"Пример отобранных признаков: {selected_feature_names[:min(10, len(selected_feature_names))]}")
elif X_selected_features.shape[1] > 10:
     print(f"Пример первых 5 отобранных признаков: {selected_feature_names[:5]}")


if X_selected_features.shape[1] == 0:
    raise ValueError("Отбор признаков не оставил ни одного признака. Проверьте параметры SelectFromModel.")
if X_selected_features.shape[1] != NUM_FEATURES_TO_SELECT:
    print(f"ВНИМАНИЕ: Отобрано {X_selected_features.shape[1]} признаков, хотя запрашивалось {NUM_FEATURES_TO_SELECT}. "
          "Это может произойти, если общее количество признаков меньше или если есть признаки с одинаковой важностью.")

X_temp, X_test, y_temp, y_test = train_test_split(
    X_selected_features, y_labels, test_size=0.3, random_state=42, stratify=y_labels
)

X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.3, random_state=42, stratify=y_temp
)

print(f"Размеры итоговых выборок:")
print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
print(f"y_train: 0={np.sum(y_train==0)}, 1={np.sum(y_train==1)}")
print(f"y_val:   0={np.sum(y_val==0)}, 1={np.sum(y_val==1)}")
print(f"y_test:  0={np.sum(y_test==0)}, 1={np.sum(y_test==1)}")

Отбор признаков из 259 до 80...
Количество признаков после отбора: 80
Пример первых 5 отобранных признаков: ['feat_price_pct_change_roll_mean4', 'feat_btc_correlation_lag2', 'feat_rsi_roll_mean8', 'feat_spread_pct_lag8', 'feat_long_liq_usd_roll_mean8']
Размеры итоговых выборок:
Train: (2499, 80), Val: (1071, 80), Test: (1531, 80)
y_train: 0=1310, 1=1189
y_val:   0=561, 1=510
y_test:  0=803, 1=728


# AI Create

## Training model

In [117]:
MODEL_SEED = 5
np.random.seed(MODEL_SEED)
tf.random.set_seed(MODEL_SEED)
print(f"ОБУЧЕНИЕ МОДЕЛИ АНСАМБЛЯ С MODEL_SEED = {MODEL_SEED}")
# -------------------------------------------------------

model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],),
                          kernel_regularizer=tf.keras.regularizers.l2(0.0015)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.35),

    tf.keras.layers.Dense(128, activation='relu',
                          kernel_regularizer=tf.keras.regularizers.l2(0.0015)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.35),

    tf.keras.layers.Dense(64, activation='relu',
                          kernel_regularizer=tf.keras.regularizers.l2(0.0015)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Dense(32, activation='relu',
                          kernel_regularizer=tf.keras.regularizers.l2(0.0015)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Dense(16, activation='relu',
                          kernel_regularizer=tf.keras.regularizers.l2(0.0015)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Dense(1, activation='sigmoid')
])

class_weight = {0: 1.0, 1: 2.5}

LEARNING_RATE = 0.001

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model_save_dir = "saved_ensemble_models"
if not os.path.exists(model_save_dir):
    os.makedirs(model_save_dir)

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=90,
        restore_best_weights=True,
        verbose=1,
        mode='max'
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=30,
        min_lr=0.00001,
        verbose=1,
        mode='min'
    )
]

print(f"🚀 Обучение модели ансамбля (SEED={MODEL_SEED})...")

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=200,
    batch_size=32,
    class_weight=class_weight,
    callbacks=callbacks,
    verbose=1
)

max_val_acc_this_run = 0
if hasattr(history, 'history') and history.history and 'val_accuracy' in history.history:
    max_val_acc_this_run = max(history.history['val_accuracy'])
print(f"Max val_accuracy для SEED={MODEL_SEED}: {max_val_acc_this_run:.4f}")

test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test accuracy для SEED={MODEL_SEED}: {test_acc:.4f}")

test_pred_proba = model.predict(X_test, verbose=0).flatten()
confident_predictions = np.sum((test_pred_proba > 0.7) | (test_pred_proba < 0.3))
print(f"Уверенные предсказания на тесте: {confident_predictions}/{len(test_pred_proba)} ({confident_predictions/len(test_pred_proba)*100:.1f}%)")

model_filename = f"ensemble_model_S{MODEL_SEED}_test_acc_{test_acc:.4f}_val_acc_{max_val_acc_this_run:.4f}.keras"
filepath_to_save = os.path.join(model_save_dir, model_filename)
print(f"Сохранение модели в {filepath_to_save} ...")
model.save(filepath_to_save)
print("Модель успешно сохранена.")
print("-" * 50)

ОБУЧЕНИЕ МОДЕЛИ АНСАМБЛЯ С MODEL_SEED = 5
🚀 Обучение модели ансамбля (SEED=5)...
Epoch 1/200
79/79 ━━━━━━━━━━━━━━━━━━━━ 8s 15ms/step - accuracy: 0.5049 - loss: 2.2234 - val_accuracy: 0.4883 - val_loss: 1.3656 - learning_rate: 0.0010
Epoch 2/200
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.4997 - loss: 1.9665 - val_accuracy: 0.4809 - val_loss: 1.3951 - learning_rate: 0.0010
Epoch 3/200
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.4825 - loss: 1.8636 - val_accuracy: 0.4837 - val_loss: 1.4103 - learning_rate: 0.0010
Epoch 4/200
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.4956 - loss: 1.8144 - val_accuracy: 0.4865 - val_loss: 1.4097 - learning_rate: 0.0010
Epoch 5/200
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.4948 - loss: 1.7618 - val_accuracy: 0.4809 - val_loss: 1.4100 - learning_rate: 0.0010
Epoch 6/200
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.4816 - loss: 1.7579 - val_accuracy: 0.4790 - val_loss: 1.4112 - learning_rate: 0.0010
Epoch 7/200


In [115]:
model_paths = [
    "saved_ensemble_models/ensemble_model_S10_test_acc_0.5996_val_acc_0.5752.keras",
    "saved_ensemble_models/ensemble_model_S3_test_acc_0.6042_val_acc_0.5798.keras",
]

all_probas = []

for model_path in model_paths:
    print(f"Загрузка и предсказание моделью: {model_path}")
    model = tf.keras.models.load_model(model_path)
    probas = model.predict(X_test, verbose=0).flatten()
    all_probas.append(probas)
    tf.keras.backend.clear_session()

stacked_probas = np.stack(all_probas, axis=0)
mean_probas = np.mean(stacked_probas, axis=0)

ensemble_preds = (mean_probas > 0.5).astype(int)

ensemble_accuracy = accuracy_score(y_test, ensemble_preds)
print(f"\nEnsemble Test Accuracy (average probability, {len(model_paths)} models): {ensemble_accuracy:.4f} ({ensemble_accuracy*100:.2f}%)")

confident_ensemble_predictions = np.sum((mean_probas > 0.7) | (mean_probas < 0.3))
print(f"Уверенные предсказания: {confident_ensemble_predictions}/{len(mean_probas)} ({confident_ensemble_predictions/len(mean_probas)*100:.1f}%)")

Загрузка и предсказание моделью: saved_ensemble_models/ensemble_model_S10_test_acc_0.5996_val_acc_0.5752.keras
Загрузка и предсказание моделью: saved_ensemble_models/ensemble_model_S3_test_acc_0.6042_val_acc_0.5798.keras
Загрузка и предсказание моделью: saved_ensemble_models/ensemble_model_S5_test_acc_0.5800_val_acc_0.5882.keras

Ensemble Test Accuracy (average probability, 3 models): 0.5872 (58.72%)
Уверенные предсказания: 1010/1531 (66.0%)


# Upload real data

## Extracting Features from JSON

In [114]:
df_real_test = pd.read_csv("fot_test_500.csv")
df_real_test.drop(columns=[col for col in df_real_test.columns if 'Unnamed:' in col], inplace=True, errors='ignore')

price_cols = ['outcome_price', 'initial_entry_price']
all_potential_feature_cols = [col for col in df_real_test.columns if col.startswith('feat_') or col.startswith('strat_') or col == 'composite_score_value']

if 'feat_market_state' in df_real_test.columns and df_real_test['feat_market_state'].dtype == 'object':
    df_real_test['feat_market_state'] = df_real_test['feat_market_state'].map({'TREND': 1, 'TRANSITION': 0.5, 'FLAT': 0}).fillna(0)

cols_to_convert_numeric = price_cols + all_potential_feature_cols
for col_name in list(set(cols_to_convert_numeric)):
    if col_name in df_real_test.columns:
        df_real_test[col_name] = pd.to_numeric(df_real_test[col_name].astype(str).str.replace(',', '.'), errors='coerce')

cols_for_dropna_real = price_cols[:]
if 'outcome_status' in df_real_test.columns: cols_for_dropna_real.append('outcome_status')
df_real_test.dropna(subset=cols_for_dropna_real, inplace=True)
if df_real_test.empty: raise ValueError("Реальный тестовый DataFrame пуст после базовой очистки.")

base_feature_columns_real = [col for col in all_potential_feature_cols if col in df_real_test.columns and pd.api.types.is_numeric_dtype(df_real_test[col])]
if 'feat_market_state' in df_real_test.columns and pd.api.types.is_numeric_dtype(df_real_test['feat_market_state']) and 'feat_market_state' not in base_feature_columns_real:
    base_feature_columns_real.append('feat_market_state')
base_feature_columns_real = list(set(base_feature_columns_real))
df_real_test[base_feature_columns_real] = df_real_test[base_feature_columns_real].fillna(0)

new_cols_data_real = {}
lags = [1, 2, 4, 8]; roll_windows = [4, 8]
for col in base_feature_columns_real:
    for lag in lags: new_cols_data_real[f'{col}_lag{lag}'] = df_real_test[col].shift(lag)
    for window in roll_windows: new_cols_data_real[f'{col}_roll_mean{window}'] = df_real_test[col].rolling(window=window, min_periods=1).mean()
df_real_test = pd.concat([df_real_test, pd.DataFrame(new_cols_data_real, index=df_real_test.index)], axis=1)
df_real_test.dropna(inplace=True)
if df_real_test.empty: raise ValueError("Реальный тестовый DataFrame пуст после Feature Engineering.")

for col_to_check in X_before_selection_columns: # X_before_selection_columns должен быть определен ранее!
    if col_to_check not in df_real_test.columns: df_real_test[col_to_check] = 0
X_real_test_prepared = df_real_test[X_before_selection_columns].fillna(0) # fillna(0) после отбора нужных колонок

# --- Шаг 2: Масштабирование и отбор признаков ---
# scaler и selector должны быть вашими обученными объектами!
X_real_test_scaled_all = scaler.transform(X_real_test_prepared)
X_real_test_scaled_all = np.nan_to_num(X_real_test_scaled_all, nan=0.0, posinf=0.0, neginf=0.0)
X_real_test_selected = selector.transform(X_real_test_scaled_all)

y_real_test = (df_real_test.loc[X_real_test_prepared.index, 'outcome_status'] == 'LONG_MET').astype(int) if 'outcome_status' in df_real_test.columns and not df_real_test.empty else None

# --- Шаг 3: Предсказание ансамблем ---
model_ensemble_paths = [ # Обновите этот список путями к вашим лучшим моделям
    "saved_ensemble_models/ensemble_model_S10_test_acc_0.5996_val_acc_0.5752.keras",
    "saved_ensemble_models/ensemble_model_S7_test_acc_0.5918_val_acc_0.5798.keras",
]
all_real_probas = []
for model_path in model_ensemble_paths:
    if not os.path.exists(model_path): print(f"Пропуск: {model_path}"); continue
    model = tf.keras.models.load_model(model_path)
    all_real_probas.append(model.predict(X_real_test_selected, verbose=0).flatten())
    tf.keras.backend.clear_session()

if not all_real_probas: raise ValueError("Модели для ансамбля не загружены.")

mean_real_probas = np.mean(np.stack(all_real_probas, axis=0), axis=0)
ensemble_real_preds = (mean_real_probas > 0.5).astype(int)

print(f"\n--- Результаты ансамбля на реальных данных ({len(X_real_test_selected)} записей) ---")
for i in range(min(5, len(X_real_test_selected))):
    pred_label = "LONG" if ensemble_real_preds[i] == 1 else "SHORT"
    actual_val = y_real_test.iloc[i] if y_real_test is not None and i < len(y_real_test) else "N/A"
    actual_label_info = f"(Реальный: {actual_val})" if y_real_test is not None else ""
    print(f"Пример {i+1}: Вероятность LONG={mean_real_probas[i]:.4f}, Предсказание={pred_label} {actual_label_info}")

if y_real_test is not None and len(y_real_test) == len(ensemble_real_preds):
    ensemble_real_accuracy = accuracy_score(y_real_test, ensemble_real_preds)
    print(f"\nEnsemble Accuracy на реальных данных: {ensemble_real_accuracy:.4f} ({ensemble_real_accuracy*100:.2f}%)")
    print("\nClassification Report на реальных данных:")
    print(classification_report(y_real_test, ensemble_real_preds, target_names=['SHORT', 'LONG'], zero_division=0))

    confident_ensemble_predictions = np.sum((mean_real_probas > 0.7) | (mean_real_probas < 0.3))
    print(f"Уверенные предсказания ансамбля: {confident_ensemble_predictions}/{len(mean_real_probas)} ({confident_ensemble_predictions/len(mean_real_probas)*100:.1f}%)")
else:
    print("\nНевозможно рассчитать точность: y_real_test отсутствует или его размер не совпадает.")

FileNotFoundError: [Errno 2] No such file or directory: 'fot_test_500.csv'

## Predict Results

In [ ]:
results = []

def convert_numpy_types(obj):
    if isinstance(obj, (np.generic,)):
        return obj.item()
    if isinstance(obj, np.ndarray): # Handle ndarray conversion
        return obj.tolist()
    if obj == np.inf or obj == -np.inf:
        return "Infinity" # Or None, or a large number string
    if np.isnan(obj):
        return None # Or "NaN"
    return obj

def format_symbol(symbol):
    if isinstance(symbol, str):
        return symbol.replace("/USDT", "").upper()
    return symbol

# Corrected print statements for X
if isinstance(X, np.ndarray):
    print(f"Форма X (NumPy array): {X.shape}")
    if X.ndim > 1:
        print(f"Количество колонок (признаков) в X: {X.shape[1]}")
    elif X.ndim == 1 and len(df) == X.shape[0]: # If X is 1D but per row for df (unlikely for features)
        print(f"X is 1D, length: {X.shape[0]}")
    elif X.ndim == 1: # If X is a single feature vector not matching df rows
         print(f"X is a single 1D feature vector, length: {X.shape[0]}")

else: # Should not happen based on the error, but good for robustness
    print("X is not a NumPy array, attempting to use .columns")
    print("Колонки в X:", list(X.columns))
    print("Количество колонок в X:", len(X.columns))


# Assuming X and df have the same number of rows
# and X contains the features for each row in df.
if len(df) != X.shape[0] and isinstance(X, np.ndarray):
    print(f"ВНИМАНИЕ: Количество строк в df ({len(df)}) не совпадает с количеством строк в X ({X.shape[0]})!")
    # Decide how to proceed or stop if this is an issue

for i, row in df.iterrows():
    try:
        # Берем данные из X для текущей строки
        # X[i] will give the i-th row if X is a 2D NumPy array
        current_row_features = X[i].reshape(1, -1)

        # Предсказание процентного изменения
        # It's assumed 'scaler_X' is fitted on data with the same structure as rows of X
        model_input_scaled = scaler_X.transform(current_row_features)
        pred_percent_change = model.predict(model_input_scaled, verbose=0).flatten()[0]

        # Обрезаем экстремальные процентные изменения
        if pred_percent_change < -50:
            pred_percent_change = -10.0
        elif pred_percent_change > 50:
            pred_percent_change = 10.0

        # Вычисляем реальное процентное изменение для сравнения
        initial_price = float(row.get('initial_entry_price', 0))
        actual_price = float(row.get('outcome_price', 0))
        actual_percent_change = ((actual_price - initial_price) / initial_price * 100) if initial_price != 0 else 0


        result = {
            "symbol": format_symbol(row.get("symbol", "UNKNOWN")),
            "predicted_change_percent": float(pred_percent_change),
            "actual_change_percent": float(actual_percent_change),
            "actual_price": float(actual_price),
            "initial_price": float(initial_price),
        }
        results.append(result)

        if (i + 1) % 100 == 0:
            print(f"Обработано: {i + 1}/{len(df)}")

    except IndexError as e:
        print(f"Ошибка IndexError в строке {i} (возможно, X короче, чем df): {e}")
        print(f"Длина df: {len(df)}, Форма X: {X.shape if isinstance(X, np.ndarray) else 'Не NumPy массив'}")
        continue # Or break, depending on how critical this is
    except Exception as e:
        print(f"Общая ошибка в строке {i}: {e}")
        continue

# Фильтруем проблемные предсказания
valid_results = []
for result in results:
    pred = result['predicted_change_percent']
    if pred is not None and not np.isnan(pred) and not np.isinf(pred):
        valid_results.append(result)

print(f"Валидных предсказаний: {len(valid_results)} из {len(results)}")

if valid_results:
    predicted_changes = [r['predicted_change_percent'] for r in valid_results]
    actual_changes = [r['actual_change_percent'] for r in valid_results]

    # Filter out potential non-numeric or problematic actual_price values before finding min/max
    valid_actual_prices = [r['actual_price'] for r in valid_results if isinstance(r['actual_price'], (int, float)) and r['actual_price'] > 0]

    print(f"Диапазон предсказанных изменений: {min(predicted_changes):.2f}% - {max(predicted_changes):.2f}%")
    print(f"Среднее предсказанное изменение: {np.mean(predicted_changes):.2f}%")
    if actual_changes: # Ensure actual_changes is not empty
        print(f"Диапазон реальных изменений: {min(actual_changes):.2f}% - {max(actual_changes):.2f}%")
    else:
        print("Нет данных для реальных изменений.")

    if valid_actual_prices:
        print(f"Диапазон реальных цен: {min(valid_actual_prices):.4f} - {max(valid_actual_prices):.4f}")
    else:
        print("Нет валидных реальных цен для отображения диапазона.")


# Сохранение
output_data = {
    "total_predictions": len(valid_results),
    "predictions": valid_results
}

with open("final_predictions.json", 'w', encoding='utf-8') as f:
    json.dump(output_data, f, indent=2, ensure_ascii=False, default=convert_numpy_types)

print(f"✅ Сохранено {len(valid_results)} предсказаний в final_predictions.json")

X is not a NumPy array, attempting to use .columns
Колонки в X: ['strat_pattern_strength', 'strat_indicator_consensus', 'strat_indicator_strength', 'feat_vol_z_1m', 'feat_vol_z_5m', 'feat_vol_z_combined', 'feat_price_pct_change', 'feat_oi_z', 'feat_long_liq_usd', 'feat_short_liq_usd', 'feat_total_liq_usd', 'feat_cvd', 'feat_spread_pct', 'feat_trade_rate_per_min', 'feat_large_trade_ratio', 'feat_ob_imbalance', 'feat_rsi', 'feat_macd_line', 'feat_macd_signal', 'feat_macd_hist', 'feat_ema_short', 'feat_ema_long', 'feat_bb_upper', 'feat_bb_middle', 'feat_bb_lower', 'feat_bb_width_norm', 'feat_obv', 'feat_adx', 'feat_atr_norm', 'feat_atr_sma_norm', 'feat_price_slope_norm', 'feat_vwap', 'feat_funding_rate', 'feat_mark_price', 'feat_index_price', 'feat_btc_correlation', 'rsi_price_interaction', 'volume_price_ratio', 'macd_signal_strength', 'liq_imbalance']
Количество колонок в X: 40
Общая ошибка в строке 0: 0
Общая ошибка в строке 1: 1
Общая ошибка в строке 2: 2
Общая ошибка в строке 3: 3
Общ